# Notebook for å opprette sqlite-database og raadata-tabell som grunnlag for visualisering med EDITH

# Laster inn pakker

In [ ]:
import pandas as pd
import sqlite3
from sqlalchemy import Table, create_engine
from sqlalchemy.sql import select
from flask_sqlalchemy import SQLAlchemy
import getpass
import cx_Oracle
import json
import os

## from models.models_delt import connect
## Connect-funksjonen ligger i den fila, kjører i stedet som funksjon

# Laster inn forhåndsdefinert config.json
#### Fila må være fylt ut med riktige parametre og ligge i samme mappe som denne notebook'en

In [ ]:
#print(os.getcwd())

with open("config.json") as config: # Laster in valg fra config.json
    config = json.load(config)

In [ ]:
# Funksjon for å opprette sqlite-database og etablere "connection". Det er viktig at filsti er satt riktig i config.json

def connect():
    engine = create_engine(f"sqlite:///{config['data']['filsti']}/edith.sqlite")
    conn = sqlite3.connect(f"{config['data']['filsti']}/edith.sqlite", timeout=15) #Må endres hvis koblingen skal være opp mot dynarev
    db = SQLAlchemy()
    return conn, engine, db


In [ ]:
conn, engine, db = connect()
#engine, db = connect()

# Kobler til dynarev / oracle
#### Skriv inn eget brukernavn

In [ ]:
bruker = getpass.getuser()[:3]
db = cx_Oracle.connect(bruker,getpass.getpass(), "DB1P")

### Laster inn informasjon om enhetene

#### Liste med alle delreg_nr

In [ ]:
liste = []
for i in config["perioder"]:
    liste = liste + [config["perioder"][i]["delreg"]]

In [ ]:
# Henter informasjon fra SFU, ikke endre noe her!

info_foretak_dict = {}
for i in liste:
    info_foretak_dict[i] = pd.read_sql(f"SELECT NAVN, ORGNR, ORG_FORM, NACE1, NACE2, ENHETS_ID, ENHETS_TYPE FROM dynarev.vw_enhet_i_delreg WHERE DELREG_NR = {i}", con = db)
    info_foretak_dict[i]["orgnrNavn"] = info_foretak_dict[i]["ORGNR"]+str(": ")+info_foretak_dict[i]["NAVN"]
#print(info_foretak_dict)

### Laster inn skjemadata for alle perioder

In [ ]:
# Kan endre navn fra FELT_ID (fra Dynarev) til noe man ønsker selv. Default er satt til VARIABEL. 
# Kan endres av ansvarlig hvis behov, men da må det også endres i models- og templates-variabler

data_skjema_dict = {}
for i in liste:
    data_skjema_dict[i] = pd.read_sql(f"SELECT ENHETS_ID, FELT_ID as VARIABEL, FELT_VERDI, DELREG_NR FROM DYNAREV.VW_SKJEMA_DATA WHERE DELREG_NR = {i} AND AKTIV=1", con = db)

In [ ]:
for i in config["perioder"]:
    data_skjema_dict[config["perioder"][i]["delreg"]][config["perioder"][i]["periode"]] = data_skjema_dict[config["perioder"][i]["delreg"]]['FELT_VERDI']
    data_skjema_dict[config["perioder"][i]["delreg"]].drop(['FELT_VERDI'], axis = 1, inplace = True)
#print(data_skjema_dict)

### Sammenkobler skjemadata og informasjon om foretak

In [ ]:
if len(data_skjema_dict) == 1:
    data_komplett = data_skjema_dict[config["perioder"]["t"]["delreg"]]
elif len(data_skjema_dict) > 1:
    data_komplett = data_skjema_dict[config["perioder"]["t"]["delreg"]]
    config["perioder"].pop("t")
    for i in config["perioder"]:
        data_komplett = pd.merge(data_komplett, data_skjema_dict[config["perioder"][i]["delreg"]][[config["perioder"][i]["periode"], "ENHETS_ID", "VARIABEL"]], on = ['ENHETS_ID', "VARIABEL"], how = 'left')
        #print(data_komplett.head())

In [ ]:
#print(os.getcwd())

with open("config.json") as config: # Laster in valg fra config.json
    config = json.load(config)

In [ ]:
data_komplett = data_komplett.dropna(subset=[config['perioder']['t']['periode']]).drop_duplicates().reset_index(drop=True)
#print(data_komplett.head())

In [ ]:
data_komplett = pd.merge(data_komplett, info_foretak_dict[config["perioder"]["t"]["delreg"]], on = ['ENHETS_ID'], how = 'left').reset_index(drop=True)
#data_komplett.head()

In [ ]:
# data_komplett = data_komplett.rename(columns = {'FELT_ID': 'Variabel'})
data_komplett.to_sql(f'{config["tabeller"]["raadata"]}', con=engine, if_exists="replace",)

### Kontrollkjøring 

In [ ]:
#Kontrollkjøring på at tabell(er) er opprettet
tabeller = pd.read_sql("SELECT name FROM sqlite_master  WHERE type='table'", con=engine)

# Skriver ut hvilke tabeller databasen inneholder
tabeller

# Kontrollkjøring for sjekk av innhold i raadata-tabell
#grunnlag = pd.read_sql("SELECT * FROM raadata", con=engine)
#grunnlag